In [2]:
#%pip install --upgrade https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.0.0-py3-none-any.whl
# %pip install  mediapipe-silicon

In [3]:

import cv2
import numpy as np #going to be used extensively for array operations
import os #helps with filepaths
from matplotlib import pyplot as plt
import time #to provide sleep times between frames being captured
import mediapipe as mp#for all the keypoints and hand detection going to be used to train our model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


In [5]:
#cv2 captures frames of the video in bgr color and mediapipe accepts rgb color so we need to convert the color
def mediaPipeDetection(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #converts bgr to rgb
    image.flags.writeable = False #image is not writable
    results = model.process(image) #making the prediction by our mediapipe model
    image.flags.writeable = True #image is writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #converts rgb to bgr
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draws left hand landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draws right hand landmarks

In [7]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [8]:
#Path for the exported data that is the numpy arrays
DATA_PATH = os.path.join('train_videos') #creating a folder for the data

#Actions that we try to detect

actions = np.array(['big_galate', 'big_nintukolli', 'big_pustaka', 'big_buddhivanta', 'big_nanage'])

#number of videos we want to record for each action
no_sequences = 30 #number of sequences for each action

#number of frames for each video
sequence_length = 30 #number of frames in each sequence

In [9]:
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'big_galate': 0,
 'big_nintukolli': 1,
 'big_pustaka': 2,
 'big_buddhivanta': 3,
 'big_nanage': 4}

In [11]:
mp_holistic = mp.solutions.holistic

# Set the media pipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_folder = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_folder):
            continue
        
        action_dir = os.path.join('Extracted_points', action)
        if not os.path.exists(action_dir):
            os.makedirs(action_dir)
        
        action_videos = os.listdir(action_folder)
        for video_idx, video in enumerate(action_videos):
            if not video.endswith('.mp4'):
                continue
            
            video_path = os.path.join(action_folder, video)
            cap = cv2.VideoCapture(video_path)
            
            # Check if the video capture was successful
            if not cap.isOpened():
                print(f"Failed to open video file: {video_path}")
                continue
            
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            print(f"Number of frames in video: {frame_count}")
            num_sequences = frame_count // sequence_length
            
            for seq in range(num_sequences):
                seq_dir = os.path.join(action_dir, str(seq))
                if not os.path.exists(seq_dir):
                    os.makedirs(seq_dir)
                
                # Loop through each frame in the sequence
                for frame_num in range(sequence_length):
                    ret, frame = cap.read()
                    
                    # Check if frame reading was successful
                    if not ret:
                        print(f"Failed to read frame {frame_num + 1} from video: {video_path}")
                        continue
                    
                    # Process the frame to detect keypoints
                    image, results = mediaPipeDetection(frame, holistic)
                    draw_landmarks(image, results)
                    keypoints = extract_keypoints(results)
                    
                    # Save keypoints as a numpy array
                    npy_path = os.path.join(seq_dir, f'{frame_num}.npy')
                    np.save(npy_path, keypoints)
                    
                    # Display the processed frame with keypoints (optional)
                    cv2.imshow('OpenCV Feed', image)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
            
            cap.release()

cv2.destroyAllWindows()


Number of frames in video: 2190
Number of frames in video: 2190
Number of frames in video: 2190
Number of frames in video: 2762
Number of frames in video: 2605


In [12]:
# Compile and save the NumPy arrays with labels
sequences, labels = [], []

for action in actions:
    action_folder = os.path.join('Extracted_points', action)

    num_sequences = len([name for name in os.listdir(action_folder) if os.path.isdir(os.path.join(action_folder, name))])
        
    
    for seq in range(num_sequences):
        seq_dir = os.path.join(action_dir, str(seq))

        if not os.path.exists(seq_dir):
            continue  # Skip if the directory doesn't exist

        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join('Extracted_points', action, str(seq), f'{frame_num}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
print(np.array(sequences).shape)
print(np.array(labels).shape)

(391, 30, 126)
(391,)


In [14]:
x = np.array(sequences)
x.shape

(391, 30, 126)

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [18]:
y_test.shape

(20, 5)

In [19]:
tf.config.set_visible_devices([], 'GPU')

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

#### GRU MODEL

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

import cv2
import numpy as np #going to be used extensively for array operations
import os #helps with filepaths
from matplotlib import pyplot as plt
import time #to provide sleep times between frames being captured
import mediapipe as mp#for all the keypoints and hand detection going to be used to train our model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


actions = np.array(['big_galate', 'big_nintukolli', 'big_pustaka', 'big_buddhivanta', 'big_nanage'])

#number of videos we want to record for each action
no_sequences = 30 #number of sequences for each action

#number of frames for each video
sequence_length = 30 #number of frames in each sequence

colors = [(245,117,16), (117,245,16), (16,117,245), (255,0,0), (16,255,0)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    if isinstance(res, np.ndarray):
        num_classes = res.shape[0]  # Get the number of classes from res
        for class_idx, prob in enumerate(res):
            # Calculate the width of the rectangle based on probability
            width = int(prob * 100)
            # Draw rectangle and put text for each class
            cv2.rectangle(output_frame, (0, 60 + class_idx * 40), (width, 90 + class_idx * 40), colors[class_idx], -1)
            cv2.putText(output_frame, actions[class_idx], (0, 85 + class_idx * 40), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 2, cv2.LINE_AA)
    else:
        # Single value case
        width = int(res * 100)
        cv2.rectangle(output_frame, (0, 60), (width, 90), colors[0], -1)
        cv2.putText(output_frame, actions[0], (0, 85), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame


#cv2 captures frames of the video in bgr color and mediapipe accepts rgb color so we need to convert the color
def mediaPipeDetection(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #converts bgr to rgb
    image.flags.writeable = False #image is not writable
    results = model.process(image) #making the prediction by our mediapipe model
    image.flags.writeable = True #image is writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #converts rgb to bgr
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draws left hand landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draws right hand landmarks

def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

# Define the GRU model
model_gru = Sequential([
    GRU(128, return_sequences=True, activation='relu', input_shape=(30, 126)),
    GRU(128, return_sequences=True, activation='relu'),
    GRU(64, return_sequences=False, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

# Compile the model
model_gru.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
# model_gru.summary()

model_gru.load_weights('action.h5')


# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)

        # Make detections
        image, results = mediaPipeDetection(frame, holistic)
        print(results)
        

        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model_gru.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [27]:
model_gru.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) 

In [28]:
model_gru.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - categorical_accuracy: 0.2639 - loss: 1.5484
Epoch 2/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - categorical_accuracy: 0.5526 - loss: 1.3207
Epoch 3/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - categorical_accuracy: 0.6991 - loss: 0.9441
Epoch 4/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - categorical_accuracy: 0.7815 - loss: 0.5791
Epoch 5/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - categorical_accuracy: 0.8600 - loss: 0.4259
Epoch 6/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - categorical_accuracy: 0.8961 - loss: 0.3229
Epoch 7/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - categorical_accuracy: 0.9169 - loss: 0.2270
Epoch 8/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - categorical_accuracy: 0.9400 - loss: 0.1787
Epoch 9/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - categorical_accuracy: 0.9132 - loss: 0.2388
Epoch 10/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - categorical_accuracy: 0.8820 - loss: 0.3662
Epoch 11/500
12/1

In [29]:
model_gru.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 30, 128)        │        98,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 30, 128)        │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 723,089 (2.76 MB)

 Trainable params: 241,029 (941.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 482,060 (1.84 MB)

In [43]:
res = model_gru.predict(np.expand_dims(sequence, axis=0))[0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


In [44]:
res.shape

(5,)

In [45]:
actions[np.argmax(res[4])]

'big_galate'

In [46]:
actions[np.argmax(y_test[4])]

'big_nanage'

In [47]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [48]:
yhat = model_gru.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


In [49]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [50]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[17,  0],
        [ 0,  3]],

       [[15,  0],
        [ 0,  5]],

       [[16,  0],
        [ 0,  4]],

       [[16,  0],
        [ 0,  4]],

       [[16,  0],
        [ 0,  4]]], dtype=int64)

In [51]:
accuracy_score(ytrue, yhat)

1.0

In [52]:
from scipy import stats

In [58]:
colors = [(245,117,16), (117,245,16), (16,117,245), (255,0,0), (16,255,0)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    if isinstance(res, np.ndarray):
        num_classes = res.shape[0]  # Get the number of classes from res
        for class_idx, prob in enumerate(res):
            # Calculate the width of the rectangle based on probability
            width = int(prob * 100)
            # Draw rectangle and put text for each class
            cv2.rectangle(output_frame, (0, 60 + class_idx * 40), (width, 90 + class_idx * 40), colors[class_idx], -1)
            cv2.putText(output_frame, actions[class_idx], (0, 85 + class_idx * 40), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (255, 255, 255), 2, cv2.LINE_AA)
    else:
        # Single value case
        width = int(res * 100)
        cv2.rectangle(output_frame, (0, 60), (width, 90), colors[0], -1)
        cv2.putText(output_frame, actions[0], (0, 85), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame


In [64]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.flip(frame,1)

        # Make detections
        image, results = mediaPipeDetection(frame, holistic)
        print(results)
        

        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model_gru.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [14]:
cv2.destroyAllWindows()

In [66]:
model_gru.save('action.h5')

In [9]:
model_gru.load_weights('action.h5')

In [11]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draws left hand landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #draws right hand landmarks

def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])